# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

   ---------------------------------------- 0.0/987.7 kB ? eta -:--:--
   ---------------------------------------- 10.2/987.7 kB ? eta -:--:--
   - ------------------------------------- 41.0/987.7 kB 667.8 kB/s eta 0:00:02
   ---- ----------------------------------- 122.9/987.7 kB 1.0 MB/s eta 0:00:01
   --------- ------------------------------ 245.8/987.7 kB 1.5 MB/s eta 0:00:01
   ------------------ --------------------- 450.6/987.7 kB 2.2 MB/s eta 0:00:01
   -------------------------------- ------- 798.7/987.7 kB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 987.7/987.7 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.8
    Uninstalling langchain-0.2.8:
      Successfully uninstalled langchain-0.2.8


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [3]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [4]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


biology_template = """你是一位知识渊博的生物学教授。
你擅长以生动有趣的方式解释生物学概念。
当你遇到复杂问题时，你会用简单的语言和例子来帮助学生理解。

这是一个问题：
{input}"""

computer_template = """你是一位经验丰富的计算机科学教授。
你擅长用清晰明了的方式解答计算机科学问题。
当你遇到难题时，你会耐心地分解问题，逐步引导学生找到解决方案。

这是一个问题：
{input}"""

chinese_template = """你是一位博学多才的汉语言文学教授。
你擅长用优美流畅的语言讲解文学作品和语言现象。
当你遇到深奥的问题时，你会用深入浅出的方式帮助学生理解其内涵。

这是一个问题：
{input}"""

In [5]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
        {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
        {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
        {
        "name": "汉语言",
        "description": "适用于回答汉语言问题",
        "prompt_template": chinese_template,
    },
]

In [6]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [7]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [8]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [9]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [10]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [11]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言: 适用于回答汉语言问题']


In [12]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言: 适用于回答汉语言问题


In [13]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [14]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言: 适用于回答汉语言问题

<< INPUT >>
{input}

<< OU

In [15]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [16]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': 'What is blackbody radiation?'}
> Finished chain.
{'input': 'What is blackbody radiation?', 'text': '\n黑体辐射是指热能辐射的一种形式，当物体受热时会发出电磁波，其中包括可见光、红外线和紫外线等。黑体辐射的特点是其能谱分布与温度有关，遵循普朗克定律和斯蒂芬-玻尔兹曼定律。它是热力学和量子力学的重要概念，在很多领域都有广泛的应用，如宇宙学、天体物理学和热力学等。'}


In [17]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': 'What is the first prime number greater than 40, such that adding one to this prime number is divisible by 3?'}
> Finished chain.
{'input': 'What is the first prime number greater than 40, such that adding one to this prime number is divisible by 3?', 'text': '\n\n首先，我们需要找到大于40的第一个质数。质数是只能被1和自身整除的数，因此我们可以从41开始依次检查每个数是否为质数。\n\n41是一个质数，但是41+1=42不符合条件，因为42除以3余数是0。接着我们检查43，43+1=44，44除以3余数也是0，因此43不符合条件。再来看47，47+1=48，48除以3余数是0，因此47也不符合条件。\n\n我们继续检查质数49，49+1=50，50除以3余数也是0，因此49不符合条件。最后我们来看53，53+1=54，54除以3余数是0，因此53符合条件。\n\n因此，第一个大于40的质数，且加一后能被3整除的数是53。答案是53。'}


In [18]:
router_chain.verbose = True

In [19]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n\n黑洞是一种天体，它具有极强的引力，甚至连光也无法逃离它的吸引力。它们是由巨大恒星的坍塌形成的，它们的质量非常大，但体积却非常小。在它的边界，也就是所谓的“事件视界”内，重力非常强大，以至于连光都无法逃脱。因此，我们无法直接观测到黑洞，只能通过它们产生的引力效应来间接证明它们的存在。'}


In [20]:
print(chain.invoke("床前明月光的下一句是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言: {'input': '床前明月光的下一句是什么？'}
> Finished chain.
{'input': '床前明月光的下一句是什么？', 'text': '\n\n疑是地上霜。'}


In [21]:
print(chain.invoke("男性的性染色体排列是？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '男性的性染色体排列是？'}
> Finished chain.
{'input': '男性的性染色体排列是？', 'text': '\n\n男性的性染色体排列是XY。在人类中，男性有一个X染色体和一个Y染色体，而女性有两个X染色体。这种性染色体排列决定了一个人的性别，而其他染色体则决定了其他特征，如身高、眼睛颜色等。这种排列也存在于其他动物物种中，但并非所有动物都有相同的性染色体排列。'}


In [22]:
print(chain.invoke("什么是补码？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '什么是补码？'}
> Finished chain.
{'input': '什么是补码？', 'text': '\n\n补码是一种用于表示负数的二进制数的编码方式。它是将负数的绝对值按位取反，再加上1所得到的数。补码的优势在于可以通过简单的加法运算来实现减法，从而简化了计算机中负数的运算。此外，补码也能够避免出现负零的情况。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains